In [3]:
!pip install hnswlib
!pip install llama-index transformers accelerate bitsandbytes

In [4]:
!pip install autollm langchain

In [5]:
!pip install kaleido cohere langchain pdfminer.six

In [1]:
!export HF_TOKEN=YOUR_TOKEN

In [1]:
from autollm.utils.document_reading import read_files_as_documents
from transformers import AutoTokenizer, AutoModel
import torch
import hnswlib
from langchain_text_splitters import RecursiveCharacterTextSplitter
import re
import numpy as np
from mlc_llm import ChatModule

In [3]:
!wget https://manuals.coolblue.nl/e0/motorola-edge-30-ultra-256gb-grijs-5g.pdf

--2024-06-03 11:14:22--  https://manuals.coolblue.nl/e0/motorola-edge-30-ultra-256gb-grijs-5g.pdf
Resolving manuals.coolblue.nl (manuals.coolblue.nl)... 108.156.83.77, 108.156.83.114, 108.156.83.98, ...
Connecting to manuals.coolblue.nl (manuals.coolblue.nl)|108.156.83.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2160720 (2,1M) [application/octet-stream]
Saving to: ‘motorola-edge-30-ultra-256gb-grijs-5g.pdf’

motorola-edge-30-ul 100%[===================>]   2,06M  1,49MB/s    in 1,4s    

2024-06-03 11:14:24 (1,49 MB/s) - ‘motorola-edge-30-ultra-256gb-grijs-5g.pdf’ saved [2160720/2160720]



In [4]:
!mkdir -p data && mv *.pdf data/

In [2]:
required_exts = [".pdf"]
documents = read_files_as_documents(input_dir="data/", required_exts=required_exts)

2024-06-08 10:32:11,492 - autollm - INFO - Reading files from data/..
Loading files: 100%|████████████████████████████| 1/1 [00:04<00:00,  4.54s/file]
2024-06-08 10:32:16,065 - autollm - INFO - Found 1 'document(s)'.


In [3]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=40,
    length_function=len,
    is_separator_regex=False,
)

In [4]:
%%timeit -n 10
texts = text_splitter.split_text(re.sub("\\n[0-9]+", "", documents[0].text))

4.46 ms ± 130 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
texts = text_splitter.split_text(re.sub("\\n[0-9]+", "", documents[0].text))
texts[0]

'User Guide\n\n\x0c© 2022 Motorola Mobility LLC. All rights reserved.'

In [6]:
tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-small-en-v1.5')
model = AutoModel.from_pretrained('BAAI/bge-small-en-v1.5')
model.eval()

/home/jgpt/miniconda3/envs/mlc-chat-venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 384, padding_idx=0)
    (position_embeddings): Embedding(512, 384)
    (token_type_embeddings): Embedding(2, 384)
    (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=384, out_features=384, bias=True)
            (key): Linear(in_features=384, out_features=384, bias=True)
            (value): Linear(in_features=384, out_features=384, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=384, out_features=384, bias=True)
            (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [7]:
# Sentences we want sentence embeddings for
sentences = ["This is bad", "This is good", "I love it", "I hate_it"]

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)
    # Perform pooling. In this case, cls pooling.
    sentence_embeddings = model_output[0][:, 0]
# normalize embeddings
sentence_embeddings = torch.nn.functional.normalize(sentence_embeddings, p=2, dim=1)
print("Sentence embeddings:", sentence_embeddings)

Sentence embeddings: tensor([[-0.0322, -0.0685, -0.0062,  ..., -0.0562,  0.0705, -0.0242],
        [-0.0956, -0.0878,  0.0068,  ...,  0.0423,  0.1237, -0.0125],
        [-0.0370, -0.0277,  0.0332,  ...,  0.0303,  0.0544, -0.0170],
        [ 0.0062, -0.0548,  0.0408,  ..., -0.0252,  0.0500, -0.0154]])


In [8]:
sentence_embeddings.shape

torch.Size([4, 384])

In [9]:
len(texts)

993

In [10]:
def embed_texts(texts):
    encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**encoded_input)
        sentence_embeddings = model_output[0][:, 0]
    sentence_embeddings = torch.nn.functional.normalize(sentence_embeddings, p=2, dim=1)
    return sentence_embeddings.numpy().astype("float32")

In [11]:
vectors = embed_texts(texts)

In [12]:
len(vectors) == len(texts)

True

In [24]:
index = hnswlib.Index(space='l2', dim=sentence_embeddings[0].shape[0])
index.init_index(max_elements=len(texts), ef_construction=64, M=48, random_seed=13)
index.set_ef(16)
index.add_items(vectors)

In [25]:
k = 3

In [26]:
labels, distances = index.knn_query(vectors, k=3)
print("Recall:", np.mean(labels[:, 0].reshape(-1) == np.arange(len(vectors))), "\n")

Recall: 0.998992950654582 



In [27]:
queries = ["How to charge?", "How to make a call?", "How to lower brightness?", 
         "How to take a screenshot?", "How do I add widgets to the home screen?"] 

In [28]:
embed_texts(queries)[0]

array([-1.50921354e-02, -5.92661127e-02, -1.35472910e-02, -1.43884206e-02,
        6.17421139e-03, -4.57389317e-02,  3.71115133e-02,  3.13305929e-02,
        2.36260779e-02,  1.98866334e-02,  1.97941009e-02,  2.75531057e-02,
       -4.39753458e-02,  4.78189997e-02,  5.42257428e-02,  6.27781376e-02,
       -1.35392202e-02, -6.17502071e-02, -7.62310401e-02,  4.48892675e-02,
        2.88034547e-02, -5.64563386e-02, -3.07297334e-02, -3.81534733e-02,
       -2.31005158e-02,  8.24700762e-03,  6.73332997e-03,  1.33294836e-02,
       -5.28629348e-02, -9.64294598e-02,  7.83366114e-02, -1.47186648e-02,
        1.16645154e-02,  3.61419022e-02,  1.49251148e-02, -3.10490169e-02,
       -3.54995430e-02, -5.14455885e-02, -1.10696824e-02,  2.42385850e-03,
        3.96867692e-02,  3.66998278e-02, -9.16260555e-02, -7.52900615e-02,
       -6.81578554e-03, -3.04054990e-02,  5.03519922e-02,  1.72017831e-02,
        1.03679195e-01,  4.96558547e-02,  3.86297852e-02,  5.77494502e-04,
       -4.96579846e-03, -

In [44]:
%%time
labels, distances = index.knn_query(embed_texts(queries), k=3)

CPU times: user 288 ms, sys: 0 ns, total: 288 ms
Wall time: 22.8 ms


In [45]:
labels

array([[586, 910, 589],
       [409, 411, 424],
       [660, 661, 265],
       [139, 140, 141],
       [199, 198,  61]], dtype=uint64)

In [46]:
q = 4
query = queries[q]
print(f"Query: {query}")
for match, dist in zip(labels[q], distances[q]):
    print('-'*35)
    print()
    print(f"Match score: {dist}")
    print(texts[match].replace("\n", "\\n"))

Query: How do I add widgets to the home screen?
-----------------------------------

Match score: 0.4159982204437256
Touch & hold a widget to add it.\n\nAdjust widget's size\n\nYou can make some widgets larger to view more content:\n.\n\nTouch & hold the widget until it's highlighted.\n. Release it to see an outline.\n. Drag a corner or side to resize.\n\nAdd app shortcuts\n.\n.\n\nSwipe up from the bottom of the home screen to open your app tray.
-----------------------------------

Match score: 0.4204174280166626
Use widgets, shortcuts, or folders\n\nAdd widgets\n\nWidgets stream information, giving some of an app’s functionality without opening the app. For example, \nthe Calendar widget shows your daily agenda and gives you one-touch access to your full calendar.\n\nAdd a widget\n.\n.\n\nTouch & hold the home screen.\n\nTouch Widgets.\n\nWidgets are listed under their related apps.\n.\n\nTouch & hold a widget to add it.
-----------------------------------

Match score: 0.4521192908

In [32]:
!ls ../dist
!ls ../dist/libs

libs  models  TinyLlama-1.1B-Chat-v1.0-MLC
TinyLlama-1.1B-Chat-v1.0-q4f32_1-vulkan.so


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [33]:
cm = ChatModule(model="../dist/TinyLlama-1.1B-Chat-v1.0-MLC", model_lib="../dist/libs/TinyLlama-1.1B-Chat-v1.0-q4f32_1-vulkan.so")

INFO:mlc_llm.support.auto_device:Not found device: cuda:0
INFO:mlc_llm.support.auto_device:Not found device: rocm:0
INFO:mlc_llm.support.auto_device:Not found device: metal:0
INFO:mlc_llm.support.auto_device:Found device: vulkan:0
INFO:mlc_llm.support.auto_device:Found device: vulkan:1
INFO:mlc_llm.support.auto_device:Found device: vulkan:2
INFO:mlc_llm.support.auto_device:Not found device: opencl:0
INFO:mlc_llm.support.auto_device:Using device: vulkan:0
INFO:mlc_llm.chat_module:Using model folder: /home/jgpt/luxai/llms/mlc-llm/dist/TinyLlama-1.1B-Chat-v1.0-MLC
INFO:mlc_llm.chat_module:Using mlc chat config: /home/jgpt/luxai/llms/mlc-llm/dist/TinyLlama-1.1B-Chat-v1.0-MLC/mlc-chat-config.json
INFO:mlc_llm.chat_module:Using library model: ../dist/libs/TinyLlama-1.1B-Chat-v1.0-q4f32_1-vulkan.so
[2024-06-03 11:31:35] INFO model_metadata.py:96: Total memory usage: 716.24 MB (Parameters: 590.24 MB. KVCache: 0.00 MB. Temporary buffer: 126.00 MB)
[2024-06-03 11:31:35] INFO model_metadata.py:10

In [34]:
print(cm.generate(query))

To add widgets to the home screen, you need to select "Widgets" in the app drawer and then tap on "Widgets" in the list. From there, you can add widgets to the home screen by swiping down on the widget you want to add and selecting "Add Widget" or "Add App" from the list. From there, you can select the widget you want to add and add it to the home screen.


In [35]:
cm.stats()

'prefill: 69.9 tok/s, decode: 159.6 tok/s'

In [43]:
cm.reset_chat()

In [37]:
def prompt_formatter(query, context_list):
    sep = " [CONTEXT_SEP] "
    return f""""Please, use the context below to anwer the query. Use only the context discard all previous knowledge except user messages.
Absorb the context as if it was not given. Answer honestly and in simple words. 
Do not necessarily use all of the context as it may contain irrelevant information.
Reply with a step by step process to fulfill the user's needs.
If you don't know the answer, simply state so. Here's the context:
{sep}{sep.join(context_list)}{sep}
Here's the query:
{query}
"""

In [47]:
%%time
context_list = list([texts[match].replace("\n", "") for match in labels[q]])
prompt = prompt_formatter(query, context_list).replace(" [CONTEXT_SEP] ", "\n\n")

CPU times: user 53 µs, sys: 12 µs, total: 65 µs
Wall time: 75.1 µs


In [48]:
print(prompt)

"Please, use the context below to anwer the query. Use only the context discard all previous knowledge except user messages.
Absorb the context as if it was not given. Answer honestly and in simple words. 
Do not necessarily use all of the context as it may contain irrelevant information.
Reply with a step by step process to fulfill the user's needs.
If you don't know the answer, simply state so. Here's the context:


Touch & hold a widget to add it.Adjust widget's sizeYou can make some widgets larger to view more content:.Touch & hold the widget until it's highlighted.. Release it to see an outline.. Drag a corner or side to resize.Add app shortcuts..Swipe up from the bottom of the home screen to open your app tray.

Use widgets, shortcuts, or foldersAdd widgetsWidgets stream information, giving some of an app’s functionality without opening the app. For example, the Calendar widget shows your daily agenda and gives you one-touch access to your full calendar.Add a widget..Touch & hold

In [49]:
print(cm.generate(prompt))

Sure, I'd be happy to help you add widgets to your home screen. Here's how to do it:

1. Touch and hold a widget on the home screen. This will bring up a pop-up menu. 2. Select "Add to Home Screen" or "Add Widget." This will add the widget to your home screen. 3. Repeat step 2 for any additional widgets you want to add. 4. Once you've added all the widgets you want to add, you can swipe up from the bottom of the home screen to reveal the app tray. 5. Drag and drop any widgets you want to move to different locations on the home screen. Remember to adjust the widget's size and orientation as needed. If you need more information or have any questions, please feel free to ask.


In [51]:
cm.stats()

'prefill: 278.3 tok/s, decode: 127.4 tok/s'

In [13]:
import pandas as pd

In [57]:
doc_vectors_df = pd.DataFrame(vectors)
doc_vectors_df

,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
0,-0.032842,-0.009280,0.055022,-0.067724,-0.065440,0.002377,0.048005,0.056406,-0.025758,-0.070270,...,0.019427,0.045021,-0.074454,-0.001336,-0.004688,0.052351,-0.024116,0.051638,-0.030786,0.073155
1,-0.023873,0.048052,0.006810,-0.053566,-0.030990,-0.028661,0.025102,0.064557,0.027655,-0.016102,...,0.002928,0.017251,-0.009519,0.015610,-0.012487,-0.035733,0.020936,0.011260,0.059333,0.072543
2,0.003668,0.009179,0.002667,-0.037524,0.005546,0.006663,0.024300,0.044484,-0.044615,0.011683,...,-0.010781,-0.002022,-0.032474,0.018512,-0.058308,-0.018376,-0.059382,-0.002678,0.095009,0.003649
3,-0.051996,-0.039681,-0.014135,-0.044073,-0.002645,-0.046867,0.023784,0.021076,-0.022028,-0.014570,...,-0.001654,-0.003693,0.004831,-0.012200,-0.015481,0.044360,-0.030640,0.002872,-0.002635,-0.035926
4,-0.020502,0.013924,0.061473,-0.073522,0.006217,-0.020361,0.032393,-0.017513,-0.060253,0.009719,...,0.011188,-0.008813,-0.037562,0.030086,-0.002388,0.038226,0.003376,0.019694,0.082184,0.005778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
988,-0.041090,0.033759,0.058024,-0.085470,-0.058646,-0.023672,0.016125,0.020488,-0.006848,-0.025886,...,0.080674,-0.050265,-0.012867,0.059968,-0.040027,0.051550,0.004089,0.048229,-0.094652,0.024754
989,-0.065883,0.023708,0.069491,-0.075818,-0.057756,0.050206,-0.029563,-0.034766,0.031167,-0.029667,...,0.017935,-0.045896,0.032228,0.066877,-0.050135,0.050063,0.001199,0.089669,-0.117983,-0.000492
990,-0.069362,0.071803,0.011254,-0.057523,-0.053804,0.028723,-0.032195,0.002902,-0.005071,-0.003162,...,0.017295,-0.036720,0.033994,0.048741,0.023198,0.020598,0.020424,0.054554,-0.094391,0.049295
991,-0.031575,-0.014701,0.007667,-0.079068,-0.099300,-0.023633,0.040763,0.035671,-0.037519,-0.066435,...,0.003909,-0.056553,0.022574,0.002882,0.040268,0.051060,0.004099,0.052169,-0.013883,0.033499


In [ ]:
embed_texts(queries)

In [125]:
index.save_index("data/document_index.bin")

In [126]:
index = hnswlib.Index(space='l2', dim=sentence_embeddings[0].shape[0])
index.init_index(max_elements=len(texts), ef_construction=64, M=48, random_seed=13)
index.set_ef(16)
index.add_items(embed_texts(queries))
index.save_index("data/queries_index.bin")

In [15]:
pd.Series(texts, name="chunks").map(lambda x: x.replace("\n", "\\n").replace("\t", "")).to_csv("data/documents1.csv", index=False, sep="\t")

In [19]:
pd.Series(texts, name="chunks").to_csv("data/documents1.csv", index=False, sep="|")

In [131]:
pd.Series(queries, name="queries").to_csv("data/queries.csv", index=False)